In [10]:
import cv2
import numpy as np

# Load the image (you can replace this with your camera frame)
def show_grid(image_path):
    image = cv2.imread(image_path)
    # Resize the image for faster processing
    image = cv2.resize(image, (600, 600))

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    blur = cv2.GaussianBlur(gray, (5,5), 0)

    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    max_area = 0
    c = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 10000:
            if area > max_area:
                max_area = area
                best_cnt = i
                image = cv2.drawContours(image, contours, c, (0, 255, 0), 3)
        c+=1

    mask = np.zeros((gray.shape),np.uint8)
    cv2.drawContours(mask,[best_cnt],0,255,-1)
    cv2.drawContours(mask,[best_cnt],0,0,2)

    out = np.zeros_like(gray)
    out[mask == 255] = gray[mask == 255]

    blur = cv2.GaussianBlur(out, (5,5), 0)

    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    c = 0
    for i in contours:
        area = cv2.contourArea(i)
        if area > 1000/2:
            cv2.drawContours(image, contours, c, (0, 255, 0), 3)
        c+=1


    cv2.imshow("Final Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [11]:
import os

folder_path = "../mgs2/"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        show_grid(file_path)

In [31]:
import cv2
import numpy as np

def detect_circles(image_path):
    # Load the image
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
  
    # Blur using 3 * 3 kernel. 
    gray_blurred = cv2.blur(gray, (5, 5)) 
    
    # Apply Hough transform on the blurred image. 
    detected_circles = cv2.HoughCircles(gray_blurred,  
                    cv2.HOUGH_GRADIENT, 1, 20, param1 = 50, 
                param2 = 30, minRadius = 1, maxRadius = 40) 
    
    # Draw circles that are detected. 
    if detected_circles is not None: 
    
        # Convert the circle parameters a, b and r to integers. 
        detected_circles = np.uint16(np.around(detected_circles)) 
    
        for pt in detected_circles[0, :]: 
            a, b, r = pt[0], pt[1], pt[2] 
    
            # Draw the circumference of the circle. 
            cv2.circle(img, (a, b), r, (0, 255, 0), 2) 
    
            # Draw a small circle (of radius 1) to show the center. 
            cv2.circle(img, (a, b), 1, (0, 0, 255), 3) 
        cv2.imshow("Detected Circle", img) 
        cv2.waitKey(0) 

In [32]:
import os

folder_path = "../mgs2/"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        detect_circles(file_path)

In [27]:
detect_circles(r'C:\Users\77019\Desktop\kbtu\5 sem\tic_tac_toe_cv\mgs2\photo_2024-10-16_15-34-23.jpg')


In [ ]:
show_grid()

In [3]:
# grid 
# no grid scenarios

no grid:
 - must be small grid inside as middle cell
 - other cell sizes are similar
 - 2 lines are parallel
 - inside of grid should be 9 cells

grid:
  - must be small grid inside as middle cell
 - other cell sizes are similar
 - 2 lines are parallel
 - inside of grid should be 9 cells 
 - 9 small grids or more combintorically 
 - 

 no grid algorithm ideas: 
  - copy parallel lines to make a grid
  - extend but only for certain amoutn, otherwise might be noise 

we need recognize line - made from a lot of lines perhaps

In [1]:
import cv2
import numpy as np

# Function to perform perspective transformation
def apply_perspective_transform(image, pts, grid_width=600, grid_height=600):
    # Destination points (the corners of a perfect 3x3 grid)
    dst_pts = np.array([[0, 0], [grid_width, 0], [grid_width, grid_height], [0, grid_height]], dtype="float32")
    
    # Compute the perspective transform matrix
    M = cv2.getPerspectiveTransform(pts, dst_pts)
    # Apply the perspective transformation to warp the image
    warped = cv2.warpPerspective(image, M, (grid_width, grid_height))
    
    return warped

# Function to detect the grid and apply perspective correction
def detect_grid_and_cells(image_path, resize_dim=(600, 600), blur_kernel=(5, 5), 
                          adaptive_thresh_block_size=11, adaptive_thresh_C=2, 
                          min_area=10000, show_image=True):
    image = cv2.imread(image_path)
    image = cv2.resize(image, resize_dim)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, blur_kernel, 0)
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, adaptive_thresh_block_size, adaptive_thresh_C)

    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    max_area = 0
    best_cnt = None
    for c in contours:
        area = cv2.contourArea(c)
        if area > min_area:  # Assuming the grid is large enough
            if area > max_area:
                max_area = area
                best_cnt = c

    # If grid found, find its four corner points
    if best_cnt is not None:
        epsilon = 0.02 * cv2.arcLength(best_cnt, True)
        approx = cv2.approxPolyDP(best_cnt, epsilon, True)

        if len(approx) == 4:
            # есть границы поля для игры
            # Sort the corner points to be top-left, top-right, bottom-right, bottom-left
            pts = np.array([point[0] for point in approx], dtype="float32")
            rect = np.zeros((4, 2), dtype="float32")
            s = pts.sum(axis=1)
            rect[0] = pts[np.argmin(s)]  # Top-left
            rect[2] = pts[np.argmax(s)]  # Bottom-right
            diff = np.diff(pts, axis=1)
            rect[1] = pts[np.argmin(diff)]  # Top-right
            rect[3] = pts[np.argmax(diff)]  # Bottom-left

            # Apply perspective transformation to correct the grid
            image = apply_perspective_transform(image, rect, resize_dim[0], resize_dim[1])

            # Now you can divide the warped grid into 9 cells
            cells = divide_into_cells((0, 0, resize_dim[0], resize_dim[1]), image)
        else:
            # если нету границы поля для игры
            x, y, w, h = cv2.boundingRect(best_cnt)
            cv2.drawContours(image, [best_cnt], -1, (0, 255, 0), 3)
            cells = divide_into_cells((x, y, w, h), image)

            print("Could not find four corner points of the grid.")

        # Optionally show the final image with cells
        if show_image:
            cv2.imshow("Final Image with Cells", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

        return cells

    else:
        print("No grid detected")

# Function to divide the grid into 9 cells (top-left and bottom-right corners)
def divide_into_cells(grid_bounding_box, image, color_cells=True, grid_size=3, transparency=0.4):
    x, y, w, h = grid_bounding_box
    cell_width = w // grid_size
    cell_height = h // grid_size

    # Define colors for each cell (only used if color_cells is True)
    colors = [
        (255, 0, 0), (0, 255, 0), (0, 0, 255),
        (255, 255, 0), (255, 0, 255), (0, 255, 255),
        (128, 0, 128), (128, 128, 0), (0, 128, 128)
    ]

    overlay = image.copy()  # Create an overlay for transparency
    alpha = transparency  # Transparency factor

    cells = []
    color_index = 0

    for i in range(grid_size):
        for j in range(grid_size):
            x0 = x + j * cell_width
            y0 = y + i * cell_height
            x1 = x0 + cell_width
            y1 = y0 + cell_height

            cell = {
                'x0': x0, 'y0': y0,  # Top-left corner
                'x1': x1, 'y1': y1   # Bottom-right corner
            }
            cells.append(cell)

            if color_cells:  # Only color the cells if the flag is True
                cell_color = colors[color_index % len(colors)]
                color_index += 1

                # Draw the cell with transparency
                cv2.rectangle(overlay, (cell['x0'], cell['y0']),
                              (cell['x1'], cell['y1']),
                              cell_color, -1)
                cv2.rectangle(overlay, (cell['x0'], cell['y0']),
                              (cell['x1'], cell['y1']),
                              (255, 255, 255), 2)

    # If coloring is enabled, blend the overlay with the original image
    if color_cells:
        cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)

    return cells


In [39]:
# import cv2
# import numpy as np

# # Load the image and detect the grid
# def show_grid(image_path):
#     image = cv2.imread(image_path)
#     # Resize the image for faster processing
#     image = cv2.resize(image, (600, 600))

#     # Convert to grayscale
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#     # Blur and apply adaptive threshold
#     blur = cv2.GaussianBlur(gray, (5, 5), 0)
#     thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

#     # Find contours
#     contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

#     max_area = 0
#     best_cnt = None
#     for c in contours:
#         area = cv2.contourArea(c)
#         if area > 10000:  # Grid area should be large
#             if area > max_area:
#                 max_area = area
#                 best_cnt = c

#     # If grid is found, find bounding box
#     if best_cnt is not None:
#         x, y, w, h = cv2.boundingRect(best_cnt)
#         cv2.drawContours(image, [best_cnt], -1, (0, 255, 0), 3)

#         # Divide into 9 cells (3x3) and color them transparently
#         cells = divide_into_cells((x, y, w, h), image)

#         # Show the image with transparent colored cells
#         cv2.imshow("Final Image with Transparent Cells", image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()

#     else:
#         print("No grid detected")

In [40]:
cells = detect_grid_and_transform(r"C:\Users\77019\Desktop\kbtu\5 sem\tic_tac_toe_cv\images\3.png")

In [5]:
image_path = r"C:\Users\77019\Desktop\kbtu\5 sem\tic_tac_toe_cv\new_data\images\test\photo_2024-10-25_23-20-43_augmented_train copy.jpg"
cells = detect_grid_and_cells(image_path)

In [6]:
cells

[{'x0': 0, 'y0': 0, 'x1': 200, 'y1': 200},
 {'x0': 200, 'y0': 0, 'x1': 400, 'y1': 200},
 {'x0': 400, 'y0': 0, 'x1': 600, 'y1': 200},
 {'x0': 0, 'y0': 200, 'x1': 200, 'y1': 400},
 {'x0': 200, 'y0': 200, 'x1': 400, 'y1': 400},
 {'x0': 400, 'y0': 200, 'x1': 600, 'y1': 400},
 {'x0': 0, 'y0': 400, 'x1': 200, 'y1': 600},
 {'x0': 200, 'y0': 400, 'x1': 400, 'y1': 600},
 {'x0': 400, 'y0': 400, 'x1': 600, 'y1': 600}]

In [9]:
folder_path = "../mgs2/"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        cells = detect_grid_and_cells(file_path)

Could not find four corner points of the grid.
Could not find four corner points of the grid.
Could not find four corner points of the grid.


In [ ]:
# identify parallel lines that make the grid
# copy parallel for each border until it reaches last point where 2 perpendecular lines are intersected (create a rectangle)
# we have a border 
# identify 9 cells by using intersections lines
# we have cells for all cases

In [5]:
import cv2
import numpy as np

def show_grid(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (600, 600))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Line detection using Hough Line Transform
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)

    # Initialize lists to store vertical and horizontal lines
    vertical_lines = []
    horizontal_lines = []

    # Separate the lines into vertical and horizontal based on their slope
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if abs(x1 - x2) < 10:  # Vertical line
            vertical_lines.append(line)
        elif abs(y1 - y2) < 10:  # Horizontal line
            horizontal_lines.append(line)
    
    # Sort lines based on their positions (y-coordinates for horizontal, x-coordinates for vertical)
    vertical_lines = sorted(vertical_lines, key=lambda line: line[0][0])
    horizontal_lines = sorted(horizontal_lines, key=lambda line: line[0][1])

    # Draw lines on the image for visualization
    for line in vertical_lines + horizontal_lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # Detect intersections (this is where grid cells are defined)
    def line_intersection(line1, line2):
        x1, y1, x2, y2 = line1[0]
        x3, y3, x4, y4 = line2[0]
        # Line equations: (x1,y1) to (x2,y2) and (x3,y3) to (x4,y4)
        denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
        if denom == 0:
            return None  # Lines are parallel
        px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denom
        py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denom
        return int(px), int(py)

    # Find intersections between vertical and horizontal lines
    intersections = []
    for v_line in vertical_lines:
        for h_line in horizontal_lines:
            intersection = line_intersection(v_line, h_line)
            if intersection is not None:
                intersections.append(intersection)
                cv2.circle(image, intersection, 5, (0, 0, 255), -1)

    # Define grid cells based on intersections (expecting 9 cells)
    if len(intersections) >= 9:
        # Sort intersections by y-coordinate (rows) then by x-coordinate (columns)
        intersections = sorted(intersections, key=lambda point: (point[1], point[0]))

        # Group into rows of three intersections each
        cells = [intersections[i:i+3] for i in range(0, len(intersections), 3)]

        # Draw the cells on the image
        for i in range(2):
            for j in range(2):
                top_left = cells[i][j]
                bottom_right = cells[i+1][j+1]
                cv2.rectangle(image, top_left, bottom_right, (255, 0, 0), 2)

    # Show final image with detected grid and cells
    cv2.imshow("Detected Grid", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        show_grid(file_path)